In [1]:
## nevermind this was a bad idea. should run in spyder with rhino mounted in a separate env with ramutils installed

# trying to see how reports load locations

from collections import OrderedDict
import json
import os
import os.path
from glob import glob
from os import path as osp

import h5py
import numpy as np
import pandas as pd

from contextlib import contextmanager
import tempfile

# import bptools
from bptools.jacksheet import read_jacksheet
from bptools.transform import SeriesTransformation
from bptools.util import standardize_label
# from classiflib import dtypes
from ptsa.data.readers import JsonIndexReader

# from ramutils.parameters import FilePaths
# from ramutils.parameters import StimParameters
# from ramutils.utils import extract_subject_montage, touch, bytes_to_str, tempdir
# from ramutils.log import get_logger
# from ramutils.constants import MTL_LOC_DICT, DK_LOC_DICT


ModuleNotFoundError: No module named 'bptools'

In [6]:
# class paths_class:
#     root = '/'
# paths = paths_class()

@contextmanager
def tempdir():
    """Create a temporary directory and remove its contents upon completion."""
    d = tempfile.mkdtemp()
    yield d
    try:
        shutil.rmtree(d)
    except:
        pass
    
def touch(path):
    """Mimics the unix ``touch`` command."""
    with open(path, 'a'):
        os.utime(path, None)

def load_pairs_from_json(subject, experiment, sessions=None, just_pairs=True,
                         rootdir='/'):
    """ Load montage information from pairs.json file

    Parameters
    ----------
    subject: str
        Subject ID
    experiment: str
        Experiment
    just_pairs: Bool
        If True, strip out any other metadata contained in the json file
    rootdir: str
        Mount point for RHINO

    Returns
    -------
    dict
        Dictionary containing metadata for all pairs in the given subjects'
        montage

    """
    json_reader = JsonIndexReader(os.path.join(rootdir,
                                               "protocols",
                                               "r1.json"))
    if sessions is not None:
        if len(sessions) == 1:
            all_pairs_paths = json_reader.aggregate_values('pairs',
                                                           subject_alias=subject,
                                                           experiment=experiment,
                                                           session=sessions[0])

            # For PS4 sessions, the experiment is listed as FR5/catFR5, so we need to look it up
            # based on original_experiment field instead
            if len(all_pairs_paths) == 0:
                all_pairs_paths = json_reader.aggregate_values('pairs',
                                                               subject_alias=subject,
                                                               original_experiment=experiment,
                                                               session=sessions[0])
        else:
            all_pairs_paths = json_reader.aggregate_values('pairs',
                                                           subject_alias=subject,
                                                           experiment=experiment)
    else:
        all_pairs_paths = json_reader.aggregate_values('pairs',
                                                       subject_alias=subject)

    if len(all_pairs_paths) == 0:
        raise RuntimeError("No pairs.json found for subject {} "
                           "and experiment {}".format(subject, experiment))

    elif len(all_pairs_paths) > 1:
        raise RuntimeError('Multiple montages are possible')

    # For simplicity, just load the first file since they *should* all be the
    # same
    bp_path = os.path.join(rootdir, list(all_pairs_paths)[0])
    with open(bp_path, 'r') as f:
        pair_data = json.load(f, object_pairs_hook=OrderedDict)

    if just_pairs:
        pair_data = extract_pairs_dict(pair_data)
        return pair_data

    # Update the subject_id not be standard format
    stored_subject = list(pair_data.keys())[0]
    if stored_subject != subject:
        pair_data[subject] = pair_data[stored_subject]
        del pair_data[stored_subject]

    return pair_data

def extract_pairs_dict(pairs):
    """ Extract a dictionary of pairs from the standard json structure

    Parameters
    ----------
    pairs: OrderedDict

    Returns
    -------
    OrderedDict
        Dictionary of pairs that will preserve ordering

    """
    keys = list(pairs.keys())
    # Handle empty dictionary case
    if len(keys) == 0:
        return pairs

    # Remove 'version' information. TODO: Make this more flexible
    if 'version' in keys:
        keys.remove('version')
    subject = keys[0]
    pairs = pairs[subject]['pairs']

    return pairs

def extract_pairs_dict(pairs):
    """ Extract a dictionary of pairs from the standard json structure

    Parameters
    ----------
    pairs: OrderedDict

    Returns
    -------
    OrderedDict
        Dictionary of pairs that will preserve ordering

    """
    keys = list(pairs.keys())
    # Handle empty dictionary case
    if len(keys) == 0:
        return pairs

    # Remove 'version' information. TODO: Make this more flexible
    if 'version' in keys:
        keys.remove('version')
    subject = keys[0]
    pairs = pairs[subject]['pairs']

    return pairs


def get_pairs(subject_id, experiment, sessions, paths):
    """Determine how we should figure out what pairs to use.

    Option 1: In the case of hardware bipolar recordings with the ENS, EEG
    data is stored in the HDF5 file which contains the Odin electrode config
    data so we can use this.

    Option 2: For monopolar recordings, we can just read the pairs.json from
    localization.

    Parameters
    ----------
    subject_id : str
        Subject ID
    experiment : str
        Experiment type
    sessions: list
        List of sessions to use
    paths : FilePaths
        Object for storing important file paths
    localization : int
        Localization number
    montage : int
        Montage number

    Returns
    -------
    all_pairs : dict
        All pairs used in the experiment.

    Notes
    -----
    This should only be used for getting pairs when building a report. For
    config generation, use generate_pairs_from_electrode_config. To use
    get_pairs, you would need to determine an open loop experiment that the
    subject completed and use that experiment instead of the experiment whose
    config file is being generated.

    """

    # Use * for session so we don't have to assume session numbers start at 0
    subject,montage = extract_subject_montage(subject_id)
    reader = JsonIndexReader(osp.join(paths.root,'protocols','r1.json'))
    print("montage.py")
    print(subject)
    print(experiment)
    print(montage)

    event_path = list(reader.aggregate_values(
        'task_events',subject=subject,montage=montage,experiment=experiment)
        )[0]
    beh_root = event_path.partition('behavioral')[0]

    eeg_dir = osp.join(beh_root,'ephys', 'current_processed', 'noreref', '*.h5')
    files = glob(eeg_dir)

    # Read HDF5 file to get pairs
    if len(files) > 0:
        filename = files[0]

        with h5py.File(filename, 'r') as hfile:
            config_str = hfile['/config_files/electrode_config'].value

        # This will create a temporary directory that is removed when the
        # program exists the scope of the 'with' statement
        with tempdir() as temp_path:
            config_path = osp.join(temp_path, 'electrode_config.csv')
            with open(config_path, 'wb') as f:
                f.write(config_str)

            paths.electrode_config_file = config_path

            # generate_pairs_from_electrode_config panics if the .bin file isn't
            # found, so we have to make sure it's there
            touch(config_path.replace('.csv', '.bin'))

            all_pairs = generate_pairs_from_electrode_config(subject_id,
                                                             experiment,
                                                             sessions,
                                                             paths)

    # No HDF5 file exists, meaning this was a monopolar recording... read
    # pairs.json instead
    else:
        all_pairs = load_pairs_from_json(subject_id,
                                         experiment,
                                         sessions=sessions,
                                         just_pairs=False,
                                         rootdir=paths.root)

    return all_pairs

def extract_subject_montage(subject_id):
    """ Extract the subject ID and montage number from the subject ID
    Parameters
    ----------
    subject_id: str
        Subject identifier
    Returns
    -------
    str:
        Subject identifier with montage information removed
    int:
        Montage number
    """
    tokens = subject_id.split('_')
    montage = 0 if len(tokens) == 1 else int(tokens[1])
    subject = tokens[0]
    return subject, montage

def generate_pairs_from_electrode_config(subject, experiment, session, paths):
    """Load and verify the validity of the Odin electrode configuration file.

    Parameters
    ----------
    subject : str
        Subject ID
    experiment: str
        Experiment
    paths: FilePaths
        Object containing common file paths used for building reports/configs

    Returns
    -------
    pairs_from_ec : dict
        Minimal pairs.json based on the electrode configuration

    Raises
    ------
    RuntimeError
        If the csv or bin file are not found

    """

    prefix, _ = os.path.splitext(paths.electrode_config_file)
    csv_filename = prefix + '.csv'
    bin_filename = prefix + '.bin'

    if not os.path.exists(csv_filename):
        raise RuntimeError("{} not found!".format(csv_filename))
    if not os.path.exists(bin_filename):
        raise RuntimeError("{} not found!".format(bin_filename))

    # Create SeriesTransformation object to determine if this is monopolar,
    # mixed-mode, or bipolar
    # FIXME: load excluded pairs

    xform = SeriesTransformation.create(csv_filename, paths.pairs)

    # Odin electrode configuration
    ec = xform.elec_conf

    # This will mimic pairs.json (but only with labels).
    pairs_dict = OrderedDict()

    # FIXME: move the following logic into bptools
    # Hardware bipolar mode
    pairs_from_ec = {}
    if not xform.monopolar_possible():
        contacts = ec.contacts_as_recarray()

        for ch in ec.sense_channels:
            anode, cathode = ch.contact, ch.ref
            aname = bytes_to_str(
                contacts[contacts.jack_box_num == anode].contact_name[0])
            cname = bytes_to_str(
                contacts[contacts.jack_box_num == cathode].contact_name[0])
            name = '{}-{}'.format(aname, cname)
            pairs_dict[name] = {
                'channel_1': anode,
                'channel_2': cathode
            }

        # Note this is different from neurorad pipeline pairs.json because
        # the electrode configuration trumps it
        pairs_from_ec = {subject: {'pairs': pairs_dict}}

    # For monopolar, fall back to pairs.json
    else:
        pairs_from_ec = load_pairs_from_json(subject,
                                             experiment,
                                             sessions=session,
                                             just_pairs=False,
                                             rootdir=paths.root)

    return pairs_from_ec

class FilePaths(object):
    """Paths to files that frequently get passed around to many tasks.
    All paths given relative to the root path but are converted to absolute
    paths on creation.
    Keyword arguments
    -----------------
    root : str
        Rhino mount point.
    dest : str
        Directory to write files to.
    pairs : str
        Path to ``pairs.json``.
    excluded_pairs : str
        Path to ``excluded_pairs.json``.
    electrode_config_file : str
        Path to electrode config file.
    area_file : str
        Path to surface area file. When generating Odin configuration files and
        not defined, the default behavior is to look in the same directory as
        the jacksheet for a file named ``area.txt``.
    data_db: str
        Path to directory where permanently-cached underlying data for
        reports should be stored. In general, this should only be specified
        when testing, otherwise the default location should be used
    """

    def __init__(self, **kwargs):
        # root is the only required kwarg
        self.root = os.path.expanduser(kwargs['root'])

        def get(key):
            return kwargs.get(key, None)

        def makepath(key):
            p = kwargs.get(key, None)
            return os.path.join(self.root, p.lstrip('/')) if p is not None else p

        self.dest = makepath('dest')
        self.pairs = makepath('pairs')
        self.excluded_pairs = makepath('excluded_pairs')
        self.electrode_config_file = makepath('electrode_config_file')
        self.area_file = makepath('area_file')
        self.data_db = makepath('data_db')

In [7]:
subject = 'R1525J'
experiment = 'catFR1'

In [8]:
paths = FilePaths(root='/')
all_pairs = get_pairs(subject,experiment,[0,1],paths)

montage.py
R1525J
catFR1
0


RuntimeError: generator raised StopIteration

In [16]:
pairs_from_json = load_pairs_from_json(subject, experiment, sessions=None, just_pairs=True,
                         rootdir='/')

# Check if mni coordinates are present, since this is the case only since
# neurorad v2.0
first_channel = list(pairs_from_json.keys())[0]
mni_available = ('mni' in pairs_from_json[first_channel]['atlases'].keys())

# Standardize labels from json so that lookup will be easier
pairs_from_json = {standardize_label(
    key): val for key, val in pairs_from_json.items()}

# If all_pairs is an ordered dict, this loop will preserve the ordering
all_pair_labels = all_pairs[subject]['pairs'].keys()
for pair in all_pair_labels:
    standardized_pair = standardize_label(pair)
    if standardized_pair not in pairs_from_json:
        # Log some warning here about not finding the contact
        continue
    channel_1 = pairs_from_json[standardized_pair]['channel_1']
    channel_2 = pairs_from_json[standardized_pair]['channel_2']
    all_pairs[subject]['pairs'][pair]['channel_1'] = str(channel_1)
    all_pairs[subject]['pairs'][pair]['channel_2'] = str(channel_2)
    # types should be same for both electrodes
    all_pairs[subject]['pairs'][pair]['type'] = pairs_from_json[standardized_pair]['type_1']
    all_pairs[subject]['pairs'][pair]['location'] = extract_atlas_location(
        pairs_from_json[standardized_pair])
    all_pairs[subject]['pairs'][pair]['label'] = pair
    all_pairs[subject]['pairs'][pair]['region'] = get_region_name(
        all_pairs[subject]['pairs'][pair]['location'].partition(' ')[-1]
    )

    if mni_available:
        all_pairs[subject]['pairs'][pair]['mni_x'] = pairs_from_json[
            standardized_pair]['atlases']['mni']['x']
        all_pairs[subject]['pairs'][pair]['mni_y'] = pairs_from_json[
            standardized_pair]['atlases']['mni']['y']
        all_pairs[subject]['pairs'][pair]['mni_z'] = pairs_from_json[
            standardized_pair]['atlases']['mni']['z']

NameError: name 'all_pairs' is not defined